# import

In [1]:
!pip install keybert
!pip install konlpy
!pip install dash

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=733f3e672f4510b6b773804bc80360d6908f88420c0cbdd0165df4435c7c496e
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=92263f97c4fae32062da2c5ea1c8d687aba4a6a251c4652e4d4b267a794d3546
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 41.8 MB/s eta 0:00:00

In [2]:
import pandas as pd
import numpy as np
import re

import math
from keybert import KeyBERT
from konlpy.tag import *
from collections import Counter
from sklearn.preprocessing import normalize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

import dash
import dash_table
from dash import html
from sklearn.decomposition import LatentDirichletAllocation

<ipython-input-2-f01fca5b0acf>:17: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [3]:
!pip install pyLDAvis
!pip install pandas==1.5.1
import pyLDAvis.lda_model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 34.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicor

# 전처리

**workshop과 career에 대해서 각각 실행**

## data load

In [4]:
employee_c=pd.read_csv("./career_crolling_result.csv")
employee_w=pd.read_csv("./workshop_crolling_result.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## preprocessing

In [5]:
employee_c=employee_c.drop(['Unnamed: 0','첨부파일'], axis=1, errors='ignore')

employee_w=employee_w.drop(['Unnamed: 0','첨부파일'], axis=1, errors='ignore')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
employee_c[['시작날짜', '마감날짜']] = employee_c['신청기간'].str.split(' ~ ', expand=True)
employee_c['시작날짜'] = pd.to_datetime(employee_c['시작날짜'])
employee_c['마감날짜'] = pd.to_datetime(employee_c['마감날짜'])
employee2_c=employee_c.drop(['신청기간'], axis=1, errors='ignore')

employee_w[['시작날짜', '마감날짜']] = employee_w['신청기간'].str.split(' ~ ', expand=True)
employee_w['시작날짜'] = pd.to_datetime(employee_w['시작날짜'])
employee_w['마감날짜'] = pd.to_datetime(employee_w['마감날짜'])
employee2_w=employee_w.drop(['신청기간'], axis=1, errors='ignore')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
employee2_c['정원'] = employee2_c['정원/대기자'].str.extract(r'정원 (\d+)명')
employee2_c['대기자'] = employee2_c['정원/대기자'].str.extract(r'대기자 (\d+)명')
employee3_c=employee2_c.drop(['정원/대기자'], axis=1, errors='ignore')

employee2_w['정원'] = employee2_w['정원/대기자'].str.extract(r'정원 (\d+)명')
employee2_w['대기자'] = employee2_w['정원/대기자'].str.extract(r'대기자 (\d+)명')
employee3_w=employee2_w.drop(['정원/대기자'], axis=1, errors='ignore')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
employee3_c['학부'] = employee3_c['신청조건'].str.extract(r'학부 : ([^\n]+)')
employee3_c['학적'] = employee3_c['신청조건'].str.extract(r'학적 : ([^\n]+)')
employee3_c['학년'] = employee3_c['신청조건'].str.extract(r'학년 : ([^\n]+)')
employee4_c=employee3_c.drop(['신청조건'],axis=1)

employee3_w['학부'] = employee3_w['신청조건'].str.extract(r'학부 : ([^\n]+)')
employee3_w['학적'] = employee3_w['신청조건'].str.extract(r'학적 : ([^\n]+)')
employee3_w['학년'] = employee3_w['신청조건'].str.extract(r'학년 : ([^\n]+)')
employee4_w=employee3_w.drop(['신청조건'],axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
employee4_c['수료 인증서'] = employee4_c['수료 인증서'].replace({'O': 1, 'X': 0})
employee4_c['수료 인증서'] = employee4_c['수료 인증서'].fillna(0)
employee4_c['수료 인증서'] = employee4_c['수료 인증서'].astype(int)

employee4_w['수료 인증서'] = employee4_w['수료 인증서'].replace({'O': 1, 'X': 0})
employee4_w['수료 인증서'] = employee4_w['수료 인증서'].fillna(0)
employee4_w['수료 인증서'] = employee4_w['수료 인증서'].astype(int)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
def split_text_by_percentage_and_numbers(input_series):
    new_df = pd.DataFrame()

    for i, text in input_series.items():
        if pd.isna(text) or isinstance(text, (int, float)):
            text = '0'  # NaN 값 처리
        else:
            text = str(text)

        parts = text.split('%')

        skills = {}
        for part in parts:
            part = part.strip()
            parts_with_numbers = re.split(r'(\d+)', part)
            parts_with_numbers = [part for part in parts_with_numbers if part.strip()]
            for j in range(0, len(parts_with_numbers), 2):
                if j + 1 < len(parts_with_numbers):  # 인덱스 에러 방지
                    skills[parts_with_numbers[j]] = int(parts_with_numbers[j+1])

        temp_df = pd.DataFrame(skills, index=[i])
        new_df = pd.concat([new_df, temp_df], axis=0)

    return new_df

# workshop4 DataFrame에서 '핵심역량 비중도' 열 사용
extracted_data_c = split_text_by_percentage_and_numbers(employee4_c['핵심역량 비중도'])
# NaN 값을 0으로 대체하고 모든 데이터 값을 정수로 변환
extracted_data_c = extracted_data_c.fillna(0).astype(int)
# 새로운 DataFrame에 합쳐서 저장
employee4_c = pd.concat([employee4_c, extracted_data_c], axis=1)
employee5_c=employee4_c.drop(['핵심역량 비중도'],axis=1)

extracted_data_w = split_text_by_percentage_and_numbers(employee4_w['핵심역량 비중도'])
extracted_data_w = extracted_data_w.fillna(0).astype(int)
employee4_w = pd.concat([employee4_w, extracted_data_w], axis=1)
employee5_w=employee4_w.drop(['핵심역량 비중도'],axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
def process_text(text):
    if isinstance(text, str):
        # '-'를 기준으로 앞 부분 제거
        processed_text = text.split('-', 1)[-1].strip()

        # 특수문자 제거
        processed_text = re.sub(r'[^\w\s]', '', processed_text)

        return processed_text
    return text

# '특강 및 워크숍 - 교육.특강' 열의 데이터 처리
employee5_c['프로그램 분류'] = employee5_c['프로그램 분류'].apply(process_text)
employee5_w['프로그램 분류'] = employee5_w['프로그램 분류'].apply(process_text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
employee5_c['인정시간'] = employee5_c['인정시간'].fillna(0)
employee5_w['인정시간'] = employee5_w['인정시간'].fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 텍스트 마이닝

### 키버트로 키워드 뽑기

In [13]:
def get_keyword(sentence):
    if pd.isnull(sentence):
        return 0
    doc = sentence
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(0,1),use_maxsum = False,top_n = 10)
    result = [x[0] for x in keywords]
    return result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
col_keyword = ['프로그램 목표', '프로그램 주요내용', '기대효과']

for col in col_keyword:
    employee5_c[col + '_키워드'] = employee5_c[col].apply(get_keyword)
    employee5_w[col + '_키워드'] = employee5_w[col].apply(get_keyword)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

### 명사 추출 -> 키워드


### 불용어

In [15]:
stop_words = ['',
 '않다',
 '되어다',
 '되다',
 '하다',
 '어떻다',
 '이렇다',
 '이다',
 '어제',
 '매일',
 '아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조금',
 '다수',
 '몇',
 '얼마',
 '지만',
 '하물며',
 '또한',
 '그러나',
 '그렇지만',
 '하지만',
 '이외에도',
 '대해 말하자면',
 '뿐이다',
 '다음에',
 '반대로',
 '반대로 말하자면',
 '이와 반대로',
 '바꾸어서 말하면',
 '바꾸어서 한다면',
 '만약',
 '그렇지않으면',
 '까악',
 '툭',
 '딱',
 '삐걱거리다',
 '보드득',
 '비걱거리다',
 '꽈당',
 '응당',
 '해야한다',
 '에 가서',
 '각',
 '각각',
 '여러분',
 '각종',
 '각자',
 '제각기',
 '하도록하다',
 '와',
 '과',
 '그러므로',
 '그래서',
 '고로',
 '한 까닭에',
 '하기 때문에',
 '거니와',
 '이지만',
 '대하여',
 '관하여',
 '관한',
 '과연',
 '실로',
 '아니나다를가',
 '생각한대로',
 '진짜로',
 '한적이있다',
 '하곤하였다',
 '하',
 '하하',
 '허허',
 '아하',
 '거바',
 '와',
 '오',
 '왜',
 '어째서',
 '무엇때문에',
 '어찌',
 '하겠는가',
 '무슨',
 '어디',
 '어느곳',
 '더군다나',
 '하물며',
 '더욱이는',
 '어느때',
 '언제',
 '야',
 '이봐',
 '어이',
 '여보시오',
 '흐흐',
 '흥',
 '휴',
 '헉헉',
 '헐떡헐떡',
 '영차',
 '여차',
 '어기여차',
 '끙끙',
 '아야',
 '앗',
 '아야',
 '콸콸',
 '졸졸',
 '좍좍',
 '뚝뚝',
 '주룩주룩',
 '솨',
 '우르르',
 '그래도',
 '또',
 '그리고',
 '바꾸어말하면',
 '바꾸어말하자면',
 '혹은',
 '혹시',
 '답다',
 '및',
 '그에 따르는',
 '때가 되어',
 '즉',
 '지든지',
 '설령',
 '가령',
 '하더라도',
 '할지라도',
 '일지라도',
 '지든지',
 '몇',
 '거의',
 '하마터면',
 '인젠',
 '이젠',
 '된바에야',
 '된이상',
 '만큼\t어찌됏든',
 '그위에',
 '게다가',
 '점에서 보아',
 '비추어 보아',
 '고려하면',
 '하게될것이다',
 '일것이다',
 '비교적',
 '좀',
 '보다더',
 '비하면',
 '시키다',
 '하게하다',
 '할만하다',
 '의해서',
 '연이서',
 '이어서',
 '잇따라',
 '뒤따라',
 '뒤이어',
 '결국',
 '의지하여',
 '기대여',
 '통하여',
 '자마자',
 '더욱더',
 '불구하고',
 '얼마든지',
 '마음대로',
 '주저하지 않고',
 '곧',
 '즉시',
 '바로',
 '당장',
 '하자마자',
 '밖에 안된다',
 '하면된다',
 '그래',
 '그렇지',
 '요컨대',
 '다시 말하자면',
 '바꿔 말하면',
 '즉',
 '구체적으로',
 '말하자면',
 '시작하여',
 '시초에',
 '이상',
 '허',
 '헉',
 '허걱',
 '바와같이',
 '해도좋다',
 '해도된다',
 '게다가',
 '더구나',
 '하물며',
 '와르르',
 '팍',
 '퍽',
 '펄렁',
 '동안',
 '이래',
 '하고있었다',
 '이었다',
 '에서',
 '로부터',
 '까지',
 '예하면',
 '했어요',
 '해요',
 '함께',
 '같이',
 '더불어',
 '마저',
 '마저도',
 '양자',
 '모두',
 '습니다',
 '가까스로',
 '하려고하다',
 '즈음하여',
 '다른',
 '다른 방면으로',
 '해봐요',
 '습니까',
 '했어요',
 '말할것도 없고',
 '무릎쓰고',
 '개의치않고',
 '하는것만 못하다',
 '하는것이 낫다',
 '매',
 '매번',
 '들',
 '모',
 '어느것',
 '어느',
 '로써',
 '갖고말하자면',
 '어디',
 '어느쪽',
 '어느것',
 '어느해',
 '어느 년도',
 '라 해도',
 '언젠가',
 '어떤것',
 '어느것',
 '저기',
 '저쪽',
 '저것',
 '그때',
 '그럼',
 '그러면',
 '요만한걸',
 '그래',
 '그때',
 '저것만큼',
 '그저',
 '이르기까지',
 '할 줄 안다',
 '할 힘이 있다',
 '너',
 '너희',
 '당신',
 '어찌',
 '설마',
 '차라리',
 '할지언정',
 '할지라도',
 '할망정',
 '할지언정',
 '구토하다',
 '게우다',
 '토하다',
 '메쓰겁다',
 '옆사람',
 '퉤',
 '쳇',
 '의거하여',
 '근거하여',
 '의해',
 '따라',
 '힘입어',
 '그',
 '다음',
 '버금',
 '두번째로',
 '기타',
 '첫번째로',
 '나머지는',
 '그중에서',
 '견지에서',
 '형식으로 쓰여',
 '입장에서',
 '위해서',
 '단지',
 '의해되다',
 '하도록시키다',
 '뿐만아니라',
 '반대로',
 '전후',
 '전자',
 '앞의것',
 '잠시',
 '잠깐',
 '하면서',
 '그렇지만',
 '다음에',
 '그러한즉',
 '그런즉',
 '남들',
 '아무거나',
 '어찌하든지',
 '같다',
 '비슷하다',
 '예컨대',
 '이럴정도로',
 '어떻게',
 '만약',
 '만일',
 '위에서 서술한바와같이',
 '인 듯하다',
 '하지 않는다면',
 '만약에',
 '무엇',
 '무슨',
 '어느',
 '어떤',
 '아래윗',
 '조차',
 '한데',
 '그럼에도 불구하고',
 '여전히',
 '심지어',
 '까지도',
 '조차도',
 '하지 않도록',
 '않기 위하여',
 '때',
 '시각',
 '무렵',
 '시간',
 '동안',
 '어때',
 '어떠한',
 '하여금',
 '네',
 '예',
 '우선',
 '누구',
 '누가 알겠는가',
 '아무도',
 '줄은모른다',
 '줄은 몰랏다',
 '하는 김에',
 '겸사겸사',
 '하는바',
 '그런 까닭에',
 '한 이유는',
 '그러니',
 '그러니까',
 '때문에',
 '그',
 '너희',
 '그들',
 '너희들',
 '타인',
 '것',
 '것들',
 '너',
 '위하여',
 '공동으로',
 '동시에',
 '하기 위하여',
 '어찌하여',
 '무엇때문에',
 '붕붕',
 '윙윙',
 '나',
 '우리',
 '엉엉',
 '휘익',
 '윙윙',
 '오호',
 '아하',
 '어쨋든',
 '만 못하다\t하기보다는',
 '차라리',
 '하는 편이 낫다',
 '흐흐',
 '놀라다',
 '상대적으로 말하자면',
 '마치',
 '아니라면',
 '쉿',
 '그렇지 않으면',
 '그렇지 않다면',
 '안 그러면',
 '아니었다면',
 '하든지',
 '아니면',
 '이라면',
 '좋아',
 '알았어',
 '하는것도',
 '그만이다',
 '어쩔수 없다',
 '하나',
 '일',
 '일반적으로',
 '일단',
 '한켠으로는',
 '오자마자',
 '이렇게되면',
 '이와같다면',
 '전부',
 '한마디',
 '한항목',
 '근거로',
 '하기에',
 '아울러',
 '하지 않도록',
 '않기 위해서',
 '이르기까지',
 '이 되다',
 '로 인하여',
 '까닭으로',
 '이유만으로',
 '이로 인하여',
 '그래서',
 '이 때문에',
 '그러므로',
 '그런 까닭에',
 '알 수 있다',
 '결론을 낼 수 있다',
 '으로 인하여',
 '있다',
 '어떤것',
 '관계가 있다',
 '관련이 있다',
 '연관되다',
 '어떤것들',
 '에 대해',
 '이리하여',
 '그리하여',
 '여부',
 '하기보다는',
 '하느니',
 '하면 할수록',
 '운운',
 '이러이러하다',
 '하구나',
 '하도다',
 '다시말하면',
 '다음으로',
 '에 있다',
 '에 달려 있다',
 '우리',
 '우리들',
 '오히려',
 '하기는한데',
 '어떻게',
 '어떻해',
 '어찌됏어',
 '어때',
 '어째서',
 '본대로',
 '자',
 '이',
 '이쪽',
 '여기',
 '이것',
 '이번',
 '이렇게말하자면',
 '이런',
 '이러한',
 '이와 같은',
 '요만큼',
 '요만한 것',
 '얼마 안 되는 것',
 '이만큼',
 '이 정도의',
 '이렇게 많은 것',
 '이와 같다',
 '이때',
 '이렇구나',
 '것과 같이',
 '끼익',
 '삐걱',
 '따위',
 '와 같은 사람들',
 '부류의 사람들',
 '왜냐하면',
 '중의하나',
 '오직',
 '오로지',
 '에 한하다',
 '하기만 하면',
 '도착하다',
 '까지 미치다',
 '도달하다',
 '정도에 이르다',
 '할 지경이다',
 '결과에 이르다',
 '관해서는',
 '여러분',
 '하고 있다',
 '한 후',
 '혼자',
 '자기',
 '자기집',
 '자신',
 '우에 종합한것과같이',
 '총적으로 보면',
 '총적으로 말하면',
 '총적으로',
 '대로 하다',
 '으로서',
 '참',
 '그만이다',
 '할 따름이다',
 '쿵',
 '탕탕',
 '쾅쾅',
 '둥둥',
 '봐',
 '봐라',
 '아이야',
 '아니',
 '와아',
 '응',
 '아이',
 '참나',
 '년',
 '월',
 '일',
 '령',
 '영',
 '일',
 '이',
 '삼',
 '사',
 '오',
 '육',
 '륙',
 '칠',
 '팔',
 '구',
 '이천육',
 '이천칠',
 '이천팔',
 '이천구',
 '하나',
 '둘',
 '셋',
 '넷',
 '다섯',
 '여섯',
 '일곱',
 '여덟',
 '아홉',
 '령',
 '영',
 '']



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 특수문자 및 불용어 제거

In [16]:
def retext(input_string):
    try:
        clean_string = re.sub('[^A-Za-z0-9가-힣]+', ' ', input_string)
    except:
        clean_string = ' '
    return clean_string

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
employee5_c['프로그램 주요내용'] = employee5_c['프로그램 주요내용'].fillna('')
employee5_c['프로그램 목표'] = employee5_c['프로그램 목표'].fillna('')
employee5_c['프로그램 주요내용_clean'] = employee5_c['프로그램 주요내용'].apply(retext)
employee5_c['프로그램 목표_clean'] = employee5_c['프로그램 목표'].apply(retext)

employee5_w['프로그램 주요내용'] = employee5_w['프로그램 주요내용'].fillna('')
employee5_w['프로그램 목표'] = employee5_w['프로그램 목표'].fillna('')
employee5_w['프로그램 주요내용_clean'] = employee5_w['프로그램 주요내용'].apply(retext)
employee5_w['프로그램 목표_clean'] = employee5_w['프로그램 목표'].apply(retext)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
def get_keyword(sentence):
    if pd.isnull(sentence):
        return 0
    okt = Okt()
    result = [x for x in okt.nouns(sentence)]
    ## 명사 중, 많이 등장한 명사만을 추출하고 싶다면 주석제거! ##
    # count = Counter(result)
    # result = count.most_common(20)
    result = [x for x in result if x not in stop_words]
    return result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
col_keyword = ['프로그램 목표_clean', '프로그램 주요내용_clean']

for col in col_keyword:
    employee5_c[col + '_명사'] = employee5_c[col].apply(get_keyword)
    employee5_w[col + '_명사'] = employee5_w[col].apply(get_keyword)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### 요약내용


In [20]:
#페이지 랭크 함수
def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)
    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R


def get_sentence(sentence_list):
    data = []
    okt = Okt()
    for s in sentence_list:
        if(s == "" or len(s) <= 2):
            continue
        temp_dict = {}
        sub_sent = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', s)
        temp_dict['sentence'] = sub_sent
        temp_dict['token_list'] = okt.morphs(sub_sent)
        temp_dict['noun_list'] = okt.nouns(sub_sent)
        data.append(temp_dict)

    df = pd.DataFrame(data)

    #유사도 matrix 생성
    similarity_matrix = []
    for i, row_i in df.iterrows():
        i_row_vec = []
        for j, row_j in df.iterrows():
            if i == j:
                i_row_vec.append(0.0)
            else:
                intersection = len(set(row_i['token_list']) & set(row_j['token_list']))
                log_i = math.log(len(set(row_i['token_list'])))
                log_j = math.log(len(set(row_j['token_list'])))
                similarity = intersection / (log_i + log_j+1)
                i_row_vec.append(similarity)
        similarity_matrix.append(i_row_vec)

    weight = np.array(similarity_matrix)

    # pagerank를 돌려서 rank matrix 반환
    R = pagerank(weight)
    # 반환된 matrix를 row 별로 sum
    R = R.sum(axis=1)
    # 값이 높은 5개의 문장 index를 반환
    indexs = R.argsort()[-5:]

    result = []
    for index in sorted(indexs):
        result.append(df['sentence'][index])
    return result

def add_info(string):
    sentence_list = string.split('\n')
    #문장 중, 길이가 10이상인 문장에 대해서만 실행
    sentence_list = [i for i in sentence_list if len(i) >= 10]
    try:
        result = get_sentence(sentence_list)
        result = ' '.join(result)
    except:
        result = ' '
    return result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
employee5_c['프로그램 주요내용_요약'] = employee5_c['프로그램 주요내용'].apply(add_info)
employee5_c['프로그램 주요내용_요약']

employee5_w['프로그램 주요내용_요약'] = employee5_w['프로그램 주요내용'].apply(add_info)
employee5_w['프로그램 주요내용_요약']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0         모든 특강 진행일 1일 전  오후 4시 접속링크 발송  비대면 온라인 교육입니...
1         모든 특강 진행일 1일 전  오후 4시 접속링크 발송  비대면 온라인 교육입니...
2         모든 특강 진행일 1일 전  오후 4시 접속링크 발송  비대면 온라인 교육입니...
3      자살예방교육Y생명지기을 통해 심리적 위기 및 자살 예방에 도움을 줄 수 있습니다 신...
4         모든 특강 진행일 1일 전  오후 4시 접속링크 발송  비대면 온라인 교육입니...
                             ...                        
694    전 일정 참가 가능한 학생만 지원 바람712 계절학기때문에 지원불가인 학생은 문의바...
695     1 2 3 4 5회 참가자 중 비수상팀 학생의 경우에는 재신청 가능수상팀은 신청불...
696     1 2 3 4회 참가자 중 비수상팀 학생의 경우에는 재신청 가능수상팀은 신청불가 ...
697    학생들의 진취적인 진로 탐색을 응원하는 한국외국어대학교 로컬리티 사업단은 꿈을 이루...
698                                                     
Name: 프로그램 주요내용_요약, Length: 699, dtype: object

#### csv


In [22]:
employee5_c.to_csv("./career_cleaning.csv",index=False)
employee5_w.to_csv("./workshop_cleaning.csv",index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
df_c=pd.read_csv("./career_cleaning.csv")
df_w=pd.read_csv("./workshop_cleaning.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
df2_c=df_c.drop(['운영조직','선정방법','비교과 캠퍼스','수료 인증서','인정시간','수강신청 취소허용','Unnamed: 19','시작날짜',
       '마감날짜', '정원', '대기자', '기대효과','기대효과_키워드','Unnamed: 2', 'index'], axis=1, errors='ignore')

df2_w=df_w.drop(['운영조직','선정방법','비교과 캠퍼스','수료 인증서','인정시간','수강신청 취소허용','Unnamed: 19','시작날짜',
       '마감날짜', '정원', '대기자', '기대효과','기대효과_키워드','Unnamed: 2', 'index'], axis=1, errors='ignore')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
threshold = 7  # 결측치가 threshold(3)개 이상인 행을 추출
df3_c=df2_c[df2_c.isnull().sum(axis=1) < threshold]
df3_w=df2_w[df2_w.isnull().sum(axis=1) < threshold]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
threshold = 5  # 결측치가 threshold(3)개 이상인 행을 추출
df3_c[df3_c.isnull().sum(axis=1) >= threshold]

df3_w[df3_w.isnull().sum(axis=1) >= threshold]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,프로그램명,프로그램 분류,신청대상,프로그램 목표,프로그램 주요내용,학부,학적,학년,지식·정보·기술 활용역량,자기주도적 학습역량,...,의사소통 역량,대인관계역량,세계시민역량,프로그램 목표_키워드,프로그램 주요내용_키워드,프로그램 주요내용_clean,프로그램 목표_clean,프로그램 목표_clean_명사,프로그램 주요내용_clean_명사,프로그램 주요내용_요약
30,2023-2 중국 유학생 학업 및 진로 특강 / 2023-2 国际学生 特别学术和职业讲座,NaN,NaN,유학생의 대학생활 적응 향상 (진로발달 및 학업동기 향상),안녕하세요. 한국외국어대학교 학생상담센터입니다.\n학생상담센터에서는 유학생들을 위한...,NaN,NaN,NaN,25,50,...,0,0,0,"['유학생의', '학업동기', '대학생활', '적응', '진로발달', '향상']","['학생상담센터입니다', '학생상담센터에서는', '안녕하세요', '学生咨询中心计划为...",안녕하세요 한국외국어대학교 학생상담센터입니다 학생상담센터에서는 유학생들을 위한 프로...,유학생의 대학생활 적응 향상 진로발달 및 학업동기 향상,"['유학생', '대학', '생활', '적응', '향상', '진로', '발달', '학...","['한국', '외국어', '대학교', '학생', '상담', '센터', '학생', '...",
33,[특교원 우즈베크어] 현지지역전문가 특강 - 타슈켄트국립동방학대학교 총장,NaN,NaN,현지 지역전문가 특강으로 인한 우즈베키스탄에 대한 관심도 재고,Tixsiyeva S.Gulchehra- 타슈켄트국립동방학대학교(TDShU) 총장 ...,NaN,NaN,NaN,0,0,...,0,0,0,"['우즈베키스탄에', '지역전문가', '특강으로', '현지', '관심도', '재고'...","['타슈켄트국립동방학대학교', '우즈베키스탄에', 'tixsiyeva', '총장의'...",Tixsiyeva S Gulchehra 타슈켄트국립동방학대학교 TDShU 총장 방문...,현지 지역전문가 특강으로 인한 우즈베키스탄에 대한 관심도 재고,"['현지', '지역', '전문가', '특강', '우즈베키스탄', '대한', '관심'...","['타슈켄트', '국립', '방학', '대학교', '총장', '방문', '현지', ...",Tixsiyeva SGulchehra 타슈켄트국립동방학대학교TDShU 총장 방문 현...
233,HUFS English Writing Tutoring 4월 2주차 (4/3-4/9),NaN,NaN,글로벌 사회와의 효과적인 소통을 위해 학습자들의 외국어 글쓰기 능력 향상,!첨삭 받을 파일(과제) 수업시간 48시간 전 제출 필수(이후 제출 시 무통보 불합...,NaN,NaN,NaN,0,50,...,50,0,0,"['글쓰기', '효과적인', '소통을', '학습자들의', '능력', '글로벌', '...","['수강과목과', '완료해주시기', '유의사항', '상황으로', '작성해주시기', ...",첨삭 받을 파일 과제 수업시간 48시간 전 제출 필수 이후 제출 시 무통보 불합격...,글로벌 사회와의 효과적인 소통을 위해 학습자들의 외국어 글쓰기 능력 향상,"['글로벌', '사회', '효과', '소통', '위해', '학습자', '외국어', ...","['삭', '파일', '과제', '수업', '전', '제출', '필수', '이후',...",신청 방법 HUFSAbility 로그인 HUFS Writing Tutorin...
263,2023 금융권 취업대비 프로젝토링 - 증권IB ECM/DCM반,NaN,NaN,금융권 채용에 대비하여 금융권 입사에 필요한 기초 지식부터 필수 역량까지 준비,상세 안내 바로가기 : https://hufsxcareerhigh.oopy.io\n...,NaN,NaN,NaN,50,25,...,25,0,0,"['금융권', '대비하여', '채용에', '지식부터', '입사에', '필요한', '...","['재학생의', '학생의', '존중해주세요', '신청바랍니다', '대상으로', '정...",상세 안내 바로가기 https hufsxcareerhigh oopy io 소규모 인...,금융권 채용에 대비하여 금융권 입사에 필요한 기초 지식부터 필수 역량까지 준비,"['금융', '채용', '대비', '금융', '입사', '기초', '지식', '필수...","['상세', '안내', '바로가기', '소규모', '인원', '운영', '주', '...",상세 안내 바로가기 httpshufsxcareerhighoopyio 소규모 인원...
284,포스트코로나 시대의 MZ세대 소비심리와 F&B 트렌드 -남민정 인사이트플랫폼 대표 특강,NaN,NaN,"저학년들의 진로 탐색을 돕고, 도전정신과 리더십을 함양",1. 프로그램명: 2022 HUFS 명사특강\n2. 강연자 약력:\n남민정 인사...,NaN,NaN,NaN,50,50,...,0,0,0,"['도전정신과', '저학년들의', '리더십을', '돕고', '탐색을', '진로', ...","['명사특강', '관광학', '관광학부', '강연자', '프로그램명', '소비심리와...",1 프로그램명 2022 HUFS 명사특강 2 강연자 약력 남민정 인사이트플랫폼 대...,저학년들의 진로 탐색을 돕고 도전정신과 리더십을 함양,"['저학년', '진로', '탐색', '도전', '정신과', '리더십', '함양']","['프로그램', '명사', '특강', '강연', '약력', '남', '민정', '인...",남민정 인사이트플랫폼 대표 인사이트플랫폼 대표 한양대학교 관광학부 겸임교수 201...
441,2022학년도 1학기 창업 Academy 특강(1회차),NaN,NaN,"창업에 대한 흥미와 도전정신 고취, 예비창업자 역량 강화, 기업가정신 발현 유도",ㅇ 프로그램명 : 2022학년도 1학기 창업 Academy 특강-1회차\nㅇ 일 ...,NaN,NaN,NaN,0,0,...,0,50,0,"['기업가정신', '도전정신', '창업에', '강화', '발현', '흥미와', '대...","['창업교육센터', '2022학년도', '엠지플레잉', '재적생', '강연자', '...",프로그램명 2022학년도 1학기 창업 Academy 특강 1회차 일 정 2022 ...,창업에 대한 흥미와 도전정신 고취 예비창업자 역량 강화 기업가정신 발현 유도,"['창업', '대한', '흥미', '도전', '정신', '고취', '예비', '창업...","['프로그램', '학년', '학기', '창업', '특강', '정', '수', '방'...",프로그램명 2022학년도 1학기 창업 Academy 특강1회차 일 정 2...
458,<영혼을 수놓은 초상> 관람,NaN,NaN,"그리스의 전통 의복을 재현한 사진을 감상하고, 그리스에 대해 알아보자",한국국제교류재단에서 한-그 수교 60주년을 맞이하여 '영혼을 수놓은 초상' 전시를 ...,NaN,NaN,NaN,50,50,...,0,0,0,"['감상하고', '재현한', '사진을', '전통', '의복을', '알아보자', '그...","['한국국제교류재단에서', '영혼을', '의복을', '재현하여', '사진으로', '...",한국국제교류재단에서 한 그 수교 60주년을 맞이하여 영혼을 수놓은 초상 전시를 개최...,그리스의 전통 의복을 재현한 사진을 감상하고 그리스에 대해 알아보자,"['그리스', '전통', '의복', '재현', '사진', '감상', '그리스', '...","['한국', '국제', '교류', '재단', '수교', '주년', '맞이', '영혼...",한국국제교류재단에서 한그 수교 60주년을 맞이하여 영혼을 수놓은 초상 전시를 개최합...
525,2021.11.28. 데이터 기반 융복합 교육과정: 자연어처리 입문 (1),NaN,NaN,인공지능의 기본 개념 등에 대해서 학습하고자 한다,1. 특강 참여 링크\n-시간: 2021년 11월 28일 15:00~18:00 ...,NaN,NaN,NaN,75,0,...,0,0,0,"['인공지능의', '학습하고자', '대해서', '한다', '등에', '기본', '개념']","['교육과정', '이름설정을', '신청한', '2일자', '신청', '6일차', '...",1 특강 참여 링크 시간 2021년 11월 28

In [27]:
df3_c.fillna('-', inplace=True)
df3_w.fillna('-', inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-27-61938b7a3b85>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_c.fillna('-', inplace=True)
<ipython-input-27-61938b7a3b85>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_w.fillna('-', inplace=True)


In [28]:
def remove_special_characters(text):
    if isinstance(text, str):
        return re.sub(r'[^\w\s]', '', text)
    return text

# '프로젝트명' 열에 특수문자 제거 적용
df3_c['프로그램명'] = df3_c['프로그램명'].apply(remove_special_characters)
df3_c['프로그램 주요내용'] = df3_c['프로그램 주요내용'].apply(remove_special_characters)

df3_w['프로그램명'] = df3_w['프로그램명'].apply(remove_special_characters)
df3_w['프로그램 주요내용'] = df3_w['프로그램 주요내용'].apply(remove_special_characters)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-28-1910ef59f9f1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_c['프로그램명'] = df3_c['프로그램명'].apply(remove_special_characters)
<ipython-input-28-1910ef59f9f1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [29]:
df3_c.to_csv("./career_cleaning_py_result.csv",index=False, encoding = 'utf-8')
df3_w.to_csv("./workshop_cleaning_py_result.csv",index=False, encoding = 'utf-8')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 추천


In [30]:
workshop = pd.read_csv('./workshop_cleaning_py_result.csv')
career = pd.read_csv('./career_cleaning_py_result.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
#workshop의 column 순서로 career순서 맞추기
career = career[workshop.columns]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
# workshop과 career dataframe concat
workshop = pd.concat([workshop, career], axis = 0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
#concat 후 인덱스 초기화
workshop = workshop.reset_index()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 추천시스템 함수

In [34]:
# 가장 비슷한 프로그램을 추천해주는 함수
def get_recommendations(df, indices,title, cosine_sim):
    # 들어간 title에 맞는 인덱스를 찾는다.
    idx = indices[title]

    #  들어간 title과 다른 프로그램들과 pairwsie similarity scores를 구한다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도 점수로 내림차순 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 자기자신을 제외한 10개 뽑기
    sim_scores = sim_scores[1:11]

    program_indices = [i[0] for i in sim_scores]

    return df.iloc[program_indices]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## LDA 함수

In [35]:
def LDA(recom_df, topic_num):
    # 프로그램명, 목표, 주요내용만 LDA 대상으로 지정
    recom_df_lda = pd.DataFrame(recom_df.apply(lambda x : x['프로그램명'] +' '+ x['프로그램 목표'] +' ' + x['프로그램 주요내용'], axis = 1))

    # count vectorize
    count2 = CountVectorizer()
    recom_vec = count2.fit_transform(recom_df_lda[0])

    # 단어 사전 만들기
    vocab = count2.vocabulary_
    index_to_word = { v : k  for k , v in vocab.items() }

    # 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
    index_to_word = { v : k  for k , v in vocab.items() }

    # 앞에서 5개의 단어 출력
    # print( { k : v  for i, (k,v) in enumerate(index_to_word.items())  if i < 5 })


    # LDA 모델링 객체 생성 (토픽 갯수를 2로 설정 : 긍정/부정)
    lda = LatentDirichletAllocation(n_components = topic_num)

    #  recom_vec를 입력하여 모델 학습
    lda.fit(recom_vec)

    # # 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽, 196개의 단어)
    # print('배열 형태 : ' , lda.components_.shape)

    # #  토픽 별로 가장 중요도가 높은 단어를 10개씩 출력
    # for idx, topic in enumerate(lda.components_):
    #     print(f"토픽유형 {idx +1} : ", [  (index_to_word[i], topic[i].round(3))  for i in topic.argsort()[:-11:-1]  ])

    return {'lda' : lda, 'recom_vec' : recom_vec, 'count_vectorizer' : count2}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
def get_topic(LDA_result):
    topic_array = LDA_result['lda'].components_

    # 단어 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
    index_to_word = {v: k for k, v in LDA_result['count_vectorizer'].vocabulary_.items()}

    # 표의 헤더와 셀 내용을 저장할 리스트 생성
    table_header = ['토픽 유형'] + [f'단어 {i + 1}' for i in range(10)]
    table_cells = []

    # 토픽 별로 가장 중요도가 높은 단어를 10개씩 표에 추가
    for idx, topic in enumerate(topic_array):
        topic_words = [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]]
        table_cells.append([f'토픽 {idx + 1}'] + [f'{word[0]} ({word[1]})' for word in topic_words])
    return table_header, table_cells

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 인터페이스 함수

In [37]:
def create_dash_app(recom_df, workshop, table_header, table_cells):
    df = pd.DataFrame(recom_df)['프로그램명']
    df = pd.merge(df,workshop[['프로그램명', '프로그램 주요내용_요약']], on='프로그램명', how='left')

    app = dash.Dash(__name__)

    table_style = {
        'fontFamily': 'Arial, sans-serif',
        'borderCollapse': 'collapse',
        'width': '100%',
        'border': '1px solid #dddddd',
        'height': 'auto',
        'margin': '10px',  # 여기서 테이블 간격 조절
    }

    # 셀 스타일 정의
    cell_style = {
        'border': '1px solid #dddddd',
        'padding': '8px',
        'textAlign': 'center',
        'height': 'auto',
    }

    # 헤더 스타일 정의
    header_style = {
        'backgroundColor': '#f2f2f2',
        'border': '1px solid #dddddd',
        'padding': '8px',
        'textAlign': 'center',
        'fontWeight': 'bold',
    }

    # 첫 번째 DataTable 레이아웃
    layout1 = html.Div([
        html.H1('LDA Topic 유형', style={
            'text-align': 'center',
            'margin': '20px',  # 여백 설정
            'background-color': 'white',  # 배경색 설정
            'color': 'black',  # 글자색 설정
            'padding': '10px',  # 내부 여백 설정
            'border-radius': '10px'}),
        dash_table.DataTable(
            id='table1',
            columns=[{'name': col, 'id': col} for col in table_header],
            data=[{table_header[j]: table_cells[i][j] for j in range(len(table_header))} for i in range(len(table_cells))],
            style_table=table_style,
            style_cell=cell_style,
            style_header=header_style
        )
    ])

    layout2 = html.Div([
        html.H1('요약정보', style={
            'text-align': 'center',
            'margin': '20px',  # 여백 설정
            'background-color': 'white',  # 배경색 설정
            'color': 'black',  # 글자색 설정
            'padding': '10px',  # 내부 여백 설정
            'border-radius': '10px'}),
        dash_table.DataTable(
            id='table2',
            columns=[{"name": i, "id": i} for i in df.columns], # 요약 내용 테이블
            data=df.to_dict('records'),
            style_table=table_style,
            style_cell=cell_style,
            style_header=header_style,
            style_data={
                'height': 'auto',
                'lineHeight': '15px'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)',
                },
                {
                    'if': {'row_index': 'even'},
                    'backgroundColor': 'white',
                },
                {
                    'if': {'row_index': 0},
                    'height': 50,  # Set the height of the first row
                },
                # Add more conditional styles as needed
            ],
        ),
    ])

    # 전체 레아웃
    app.layout = html.Div([layout1, layout2], style={'margin': '20px'})

    if __name__ == '__main__':
        app.run_server(debug=True)

    return app

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 프로그램 주요내용으로 추천

In [38]:
# TF-IDF 정의
tfidf = TfidfVectorizer()

#만약 nan값이 있다면 빈 문자열로 대체
workshop['프로그램 주요내용'] = workshop['프로그램 주요내용'].fillna('')

#프로그램 주요내용으로 matrix 생성
tfidf_matrix = tfidf.fit_transform(workshop['프로그램 주요내용'])

tfidf_matrix.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1383, 13593)

In [39]:
# linear_kernel 함수를 이용하여 코사인 유사도를 구한다.
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
indices = pd.Series(workshop.index, index=workshop['프로그램명']).drop_duplicates()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
target = workshop.iloc[20]

print('프로그램명 : ',target['프로그램명'])
print('추천 목록: ')
recom_df = get_recommendations(workshop, indices, target['프로그램명'], cosine_sim)
print(recom_df['프로그램명'])

프로그램명 :  직무 부트캠프GMP 전체를 보는 눈과 핵심 개념으로 제약공장 품질QAQC업무알기
추천 목록: 
23           직무 부트캠프무역 프로세스 완벽 이해하기  서류 작성부터 클레임 대처까지
22               직무 부트캠프GMP환경 학교에선 알려주지 않는 환경 실무 이해하기
24        직무 부트캠프10년차 현직 PD와 함께 방송 영상 콘텐츠 기획제작연출 이해하기
21     직무 부트캠프이것이 찐현실 콘텐츠 마케팅 실무 5주 완성 병아리 콘텐츠 마케터 캠프
166             하계방학 대비 해외영업 직무 부트캠프제조업 B2B 해외영업 프로세스
165                하계방학 대비 해외영업 직무 부트캠프해외영업 시장분석 전략수립
703                  2023 용인기업분석 경진대회 참가팀 모집1인 참가도 가능
217       HUFS English Writing Tutoring 4월 3주차 410416
888                          2023 상반기 대기업 공채 대비 취업컨설팅
887                          2023 상반기 공기업 공채 대비 취업컨설팅
Name: 프로그램명, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
LDA_result = LDA(recom_df, topic_num = 4)
table_header, table_cells = get_topic(LDA_result)
create_dash_app(recom_df, workshop, table_header, table_cells)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<IPython.core.display.Javascript object>

In [43]:
# LDA 토픽 모델링 결과를 시각화
pyLDAvis.enable_notebook
visualization = pyLDAvis.lda_model.prepare(LDA_result['lda'], LDA_result['recom_vec'], LDA_result['count_vectorizer'])
pyLDAvis.display(visualization)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



## data soup를 이용한 추천

In [44]:
#혹시 있을지 모를 결측치 대체
workshop['프로그램 주요내용_키워드'] = workshop['프로그램 주요내용_키워드'].fillna('')
workshop['프로그램 목표_키워드'] = workshop['프로그램 목표_키워드'].fillna('')

workshop['프로그램 주요내용_clean_명사'] = workshop['프로그램 주요내용_clean_명사'].fillna('')
workshop['프로그램 목표_clean_명사'] = workshop['프로그램 목표_clean_명사'].fillna('')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [45]:
def create_soup(x, weight):
    # 가중치 설정
    cols = ['프로그램 분류', '학년', '학부', '학적'] * weight[0]
    col_nouns = [ '프로그램 주요내용_clean_명사', '프로그램 목표_clean_명사'] * weight[1]
    col_keywords = [ '프로그램 주요내용_키워드', '프로그램 목표_키워드'] * weight[2]
    col_skill = ['지식·정보·기술 활용역량', '자기주도적 학습역량', '창의·혁신역량', '의사소통 역량', '대인관계역량', '세계시민역량'] * weight[3]
    result = ''

    for cn in col_nouns:
        result = result + x[cn] + ' '
    for ck in col_keywords:
        ck = ck.split(',')
        result = result + ' '.join(x[ck]) + ' '
    for cs in col_skill:
        result = result + str(x[cs]) + ' '
    for c in cols:
        result = result + x[c] + ' '
    return result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



### 키워드 기반 추천

#### 모든 가중치가 동일

In [46]:
workshop['soup'] = workshop.apply(create_soup, weight = [1,0,1,1], axis=1)
count = CountVectorizer()
count_matrix = count.fit_transform(workshop['soup'])
indices = pd.Series(workshop.index, index=workshop['프로그램명'])
# cosine 유사도 구하기
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [47]:
target = workshop.iloc[20]

print('프로그램명 : ',target['프로그램명'])
print('추천 목록: ')
recom_df = get_recommendations(workshop, indices, target['프로그램명'], cosine_sim2)
print(recom_df['프로그램명'])

프로그램명 :  직무 부트캠프GMP 전체를 보는 눈과 핵심 개념으로 제약공장 품질QAQC업무알기
추천 목록: 
21      직무 부트캠프이것이 찐현실 콘텐츠 마케팅 실무 5주 완성 병아리 콘텐츠 마케터 캠프
22                직무 부트캠프GMP환경 학교에선 알려주지 않는 환경 실무 이해하기
24         직무 부트캠프10년차 현직 PD와 함께 방송 영상 콘텐츠 기획제작연출 이해하기
23            직무 부트캠프무역 프로세스 완벽 이해하기  서류 작성부터 클레임 대처까지
165                 하계방학 대비 해외영업 직무 부트캠프해외영업 시장분석 전략수립
166              하계방학 대비 해외영업 직무 부트캠프제조업 B2B 해외영업 프로세스
1114                         졸업생을 위한 일반기업 자기소개서 전략 컨설팅
1115                          졸업생을 위한 공기업 자기소개서 전략 컨설팅
610                 97 내 인생룸메가 되어줘 기숙사or자취생 라이프코칭 프로그램
672                                        2021 열린 상담실
Name: 프로그램명, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [48]:
LDA_result = LDA(recom_df, topic_num = 4)
table_header, table_cells = get_topic(LDA_result)
create_dash_app(recom_df, workshop, table_header, table_cells)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



<IPython.core.display.Javascript object>

In [49]:
# LDA 토픽 모델링 결과를 시각화
pyLDAvis.enable_notebook
visualization = pyLDAvis.lda_model.prepare(LDA_result['lda'], LDA_result['recom_vec'], LDA_result['count_vectorizer'])
pyLDAvis.display(visualization)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



#### 목표와 프로그램에 가중치를 더 준 경우

In [50]:
workshop['soup'] = workshop.apply(create_soup, weight = [1,0,2,1], axis=1)
count = CountVectorizer()
count_matrix = count.fit_transform(workshop['soup'])
indices = pd.Series(workshop.index, index=workshop['프로그램명'])
# cosine 유사도 구하기
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [51]:
target = workshop.iloc[20]

print('프로그램명 : ',target['프로그램명'])
print('추천 목록: ')
recom_df = get_recommendations(workshop, indices, target['프로그램명'], cosine_sim2)
print(recom_df['프로그램명'])

프로그램명 :  직무 부트캠프GMP 전체를 보는 눈과 핵심 개념으로 제약공장 품질QAQC업무알기
추천 목록: 
21      직무 부트캠프이것이 찐현실 콘텐츠 마케팅 실무 5주 완성 병아리 콘텐츠 마케터 캠프
22                직무 부트캠프GMP환경 학교에선 알려주지 않는 환경 실무 이해하기
24         직무 부트캠프10년차 현직 PD와 함께 방송 영상 콘텐츠 기획제작연출 이해하기
23            직무 부트캠프무역 프로세스 완벽 이해하기  서류 작성부터 클레임 대처까지
165                 하계방학 대비 해외영업 직무 부트캠프해외영업 시장분석 전략수립
166              하계방학 대비 해외영업 직무 부트캠프제조업 B2B 해외영업 프로세스
1335                                        면접을 부탁해930
1114                         졸업생을 위한 일반기업 자기소개서 전략 컨설팅
1115                          졸업생을 위한 공기업 자기소개서 전략 컨설팅
1370                  2021 선배를 JOB아라 연장공고 89일까지 신청서 마감
Name: 프로그램명, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [52]:
LDA_result = LDA(recom_df, topic_num = 4)
table_header, table_cells = get_topic(LDA_result)
create_dash_app(recom_df, workshop, table_header, table_cells)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



<IPython.core.display.Javascript object>

In [53]:
# LDA 토픽 모델링 결과를 시각화
pyLDAvis.enable_notebook
visualization = pyLDAvis.lda_model.prepare(LDA_result['lda'], LDA_result['recom_vec'], LDA_result['count_vectorizer'])
pyLDAvis.display(visualization)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



#### 목표와 프로그램만 포함 시킨 경우

In [54]:
workshop['soup'] = workshop.apply(create_soup, weight = [0,0,1,0], axis=1)
count = CountVectorizer()
count_matrix = count.fit_transform(workshop['soup'])
indices = pd.Series(workshop.index, index=workshop['프로그램명'])
# cosine 유사도 구하기
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [55]:
target = workshop.iloc[20]

print('프로그램명 : ',target['프로그램명'])
print('추천 목록: ')
recom_df = get_recommendations(workshop, indices, target['프로그램명'], cosine_sim2)
print(recom_df['프로그램명'])

프로그램명 :  직무 부트캠프GMP 전체를 보는 눈과 핵심 개념으로 제약공장 품질QAQC업무알기
추천 목록: 
21        직무 부트캠프이것이 찐현실 콘텐츠 마케팅 실무 5주 완성 병아리 콘텐츠 마케터 캠프
22                  직무 부트캠프GMP환경 학교에선 알려주지 않는 환경 실무 이해하기
24           직무 부트캠프10년차 현직 PD와 함께 방송 영상 콘텐츠 기획제작연출 이해하기
23              직무 부트캠프무역 프로세스 완벽 이해하기  서류 작성부터 클레임 대처까지
165                   하계방학 대비 해외영업 직무 부트캠프해외영업 시장분석 전략수립
166                하계방학 대비 해외영업 직무 부트캠프제조업 B2B 해외영업 프로세스
502                1220 해외영업무역 KT 졸업생을 위한 온라인 현직자 직무 클래스
503    1228 인사교육HRD 네이버웹툰 Developer Relations 졸업생을 위한...
504        1223 공사비영리 대한적십자사 국제협력 졸업생을 위한 온라인 현직자 직무 클래스
505               1222 마케팅MD 쿠팡이츠 졸업생을 위한 온라인 현직자 직무 클래스
Name: 프로그램명, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [56]:
LDA_result = LDA(recom_df, topic_num = 4)
table_header, table_cells = get_topic(LDA_result)
create_dash_app(recom_df, workshop, table_header, table_cells)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



<IPython.core.display.Javascript object>

In [57]:
# LDA 토픽 모델링 결과를 시각화
pyLDAvis.enable_notebook
visualization = pyLDAvis.lda_model.prepare(LDA_result['lda'], LDA_result['recom_vec'], LDA_result['count_vectorizer'])
pyLDAvis.display(visualization)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



### 명사기반 추천

#### 모든 가중치가 동일한 경우

In [58]:
workshop['soup'] = workshop.apply(create_soup, weight = [1,1,0,1], axis=1)
count = CountVectorizer()
count_matrix = count.fit_transform(workshop['soup'])
indices = pd.Series(workshop.index, index=workshop['프로그램명'])
# cosine 유사도 구하기
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [59]:
target = workshop.iloc[20]

print('프로그램명 : ',target['프로그램명'])
print('추천 목록: ')
recom_df = get_recommendations(workshop, indices, target['프로그램명'], cosine_sim2)
print(recom_df['프로그램명'])

프로그램명 :  직무 부트캠프GMP 전체를 보는 눈과 핵심 개념으로 제약공장 품질QAQC업무알기
추천 목록: 
23            직무 부트캠프무역 프로세스 완벽 이해하기  서류 작성부터 클레임 대처까지
22                직무 부트캠프GMP환경 학교에선 알려주지 않는 환경 실무 이해하기
24         직무 부트캠프10년차 현직 PD와 함께 방송 영상 콘텐츠 기획제작연출 이해하기
21      직무 부트캠프이것이 찐현실 콘텐츠 마케팅 실무 5주 완성 병아리 콘텐츠 마케터 캠프
165                 하계방학 대비 해외영업 직무 부트캠프해외영업 시장분석 전략수립
166              하계방학 대비 해외영업 직무 부트캠프제조업 B2B 해외영업 프로세스
1206                              2021 현직자 직무 부트 캠프코멘토
1325                   온라인 직무톡톡 직무특강 프로그램7차  MD 직무의 이해
1327                 온라인 직무톡톡 직무특강 프로그램5차  상품기획 직무의 이해
1328                 온라인 직무톡톡 직무특강 프로그램6차  IT산업 직무의 이해
Name: 프로그램명, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [60]:
LDA_result = LDA(recom_df, topic_num = 4)
table_header, table_cells = get_topic(LDA_result)
create_dash_app(recom_df, workshop, table_header, table_cells)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



<IPython.core.display.Javascript object>

In [61]:
# LDA 토픽 모델링 결과를 시각화
pyLDAvis.enable_notebook
visualization = pyLDAvis.lda_model.prepare(LDA_result['lda'], LDA_result['recom_vec'], LDA_result['count_vectorizer'])
pyLDAvis.display(visualization)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



#### 목표와 프로그램에 가중치 를 더 준 경우

In [62]:
workshop['soup'] = workshop.apply(create_soup, weight = [1,2,0,1], axis=1)
count = CountVectorizer()
count_matrix = count.fit_transform(workshop['soup'])
indices = pd.Series(workshop.index, index=workshop['프로그램명'])
# cosine 유사도 구하기
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [63]:
target = workshop.iloc[20]

print('프로그램명 : ',target['프로그램명'])
print('추천 목록: ')
recom_df = get_recommendations(workshop, indices, target['프로그램명'], cosine_sim2)
print(recom_df['프로그램명'])

프로그램명 :  직무 부트캠프GMP 전체를 보는 눈과 핵심 개념으로 제약공장 품질QAQC업무알기
추천 목록: 
23            직무 부트캠프무역 프로세스 완벽 이해하기  서류 작성부터 클레임 대처까지
22                직무 부트캠프GMP환경 학교에선 알려주지 않는 환경 실무 이해하기
24         직무 부트캠프10년차 현직 PD와 함께 방송 영상 콘텐츠 기획제작연출 이해하기
21      직무 부트캠프이것이 찐현실 콘텐츠 마케팅 실무 5주 완성 병아리 콘텐츠 마케터 캠프
165                 하계방학 대비 해외영업 직무 부트캠프해외영업 시장분석 전략수립
166              하계방학 대비 해외영업 직무 부트캠프제조업 B2B 해외영업 프로세스
1206                              2021 현직자 직무 부트 캠프코멘토
894                     2022 현직자 직무 부트캠프코멘토_한국외대 전용 개설
791                                 자소서LAB 2 _ 직무역량 작성
1325                   온라인 직무톡톡 직무특강 프로그램7차  MD 직무의 이해
Name: 프로그램명, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [64]:
LDA_result = LDA(recom_df, topic_num = 4)
table_header, table_cells = get_topic(LDA_result)
create_dash_app(recom_df, workshop, table_header, table_cells)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



<IPython.core.display.Javascript object>

In [65]:
# LDA 토픽 모델링 결과를 시각화
pyLDAvis.enable_notebook
visualization = pyLDAvis.lda_model.prepare(LDA_result['lda'], LDA_result['recom_vec'], LDA_result['count_vectorizer'])
pyLDAvis.display(visualization)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



#### 목표와 프로그램만 포함 시킨 경우

In [66]:
workshop['soup'] = workshop.apply(create_soup, weight = [0,1,0,0], axis=1)
count = CountVectorizer()
count_matrix = count.fit_transform(workshop['soup'])
indices = pd.Series(workshop.index, index=workshop['프로그램명'])
# cosine 유사도 구하기
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [67]:
target = workshop.iloc[20]

print('프로그램명 : ',target['프로그램명'])
print('추천 목록: ')
recom_df = get_recommendations(workshop, indices, target['프로그램명'], cosine_sim2)
print(recom_df['프로그램명'])

프로그램명 :  직무 부트캠프GMP 전체를 보는 눈과 핵심 개념으로 제약공장 품질QAQC업무알기
추천 목록: 
23            직무 부트캠프무역 프로세스 완벽 이해하기  서류 작성부터 클레임 대처까지
22                직무 부트캠프GMP환경 학교에선 알려주지 않는 환경 실무 이해하기
24         직무 부트캠프10년차 현직 PD와 함께 방송 영상 콘텐츠 기획제작연출 이해하기
21      직무 부트캠프이것이 찐현실 콘텐츠 마케팅 실무 5주 완성 병아리 콘텐츠 마케터 캠프
165                 하계방학 대비 해외영업 직무 부트캠프해외영업 시장분석 전략수립
166              하계방학 대비 해외영업 직무 부트캠프제조업 B2B 해외영업 프로세스
1206                              2021 현직자 직무 부트 캠프코멘토
894                     2022 현직자 직무 부트캠프코멘토_한국외대 전용 개설
791                                 자소서LAB 2 _ 직무역량 작성
896                   2022 동계방학 4차산업혁명 역량강화프로그램 파이썬SQL
Name: 프로그램명, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [68]:
LDA_result = LDA(recom_df, topic_num = 4)
table_header, table_cells = get_topic(LDA_result)
create_dash_app(recom_df, workshop, table_header, table_cells)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



<IPython.core.display.Javascript object>

In [69]:
# LDA 토픽 모델링 결과를 시각화
pyLDAvis.enable_notebook
visualization = pyLDAvis.lda_model.prepare(LDA_result['lda'], LDA_result['recom_vec'], LDA_result['count_vectorizer'])
pyLDAvis.display(visualization)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

